In [51]:
import numpy as np
import pandas as pd
import musicbrainzngs
import numpy as np
import csv
import matplotlib as plt
import matplotlib.pyplot as plt
import matplotlib
import json
%matplotlib inline
import graphlab

ImportError: No module named graphlab

#### Prepare data

In [2]:
train_file = 'train.csv'
test_file  = 'test.csv'

In [6]:
df_train = pd.DataFrame(pd.read_csv(train_file))
train_users = df_train.user.values
print len(train_users)
unique_train_users = list(set(train_users))
print len(unique_train_users)

4154804
233286


In [4]:
train_artists = df_train.artist.values
print len(train_artists)
unique_train_artists = list(set(train_artists))
print len(unique_train_artists)

4154804
2000


In [9]:
df_train.head()

,user,artist,plays
0,eb1c57ddc9e0e2d005169d3a1a96e8dd95e3af03,5a8e07d5-d932-4484-a7f7-e700793a9c94,554
1,44ce793a6cd9d20f13f4a576a818ef983314bb5d,a3a92047-be1c-4f3e-8960-c4f8570984df,81
2,da9cf3f557161d54b76f24db64be9cc76db008e3,eeb1195b-f213-4ce1-b28c-8565211f8e43,708
3,8fa49ab25d425edcf05d44bfc1d5aea895287d81,a1419808-65d3-4d40-998c-1a0bac65eabc,265
4,b85fcaef67d2669cd99b334b5e8c8705263db2cf,a3cb23fc-acd3-4ce0-8f36-1e5aa6a18432,220


In [7]:
users_map_dict = dict(zip(unique_train_users, np.arange(len(unique_train_users)))) 
artists_map_dict = dict(zip(unique_train_artists, np.arange(len(unique_train_artists)))) 

In [42]:
train_mat = np.zeros((len(unique_train_users), len(unique_train_artists)))
reference_mat = np.zeros((len(unique_train_users), len(unique_train_artists)))

#### Populate data

In [43]:
for index, row in df_train.iterrows():
    train_mat[users_map_dict[row["user"]], artists_map_dict[row["artist"]]] = row["plays"]
    reference_mat[users_map_dict[row["user"]], artists_map_dict[row["artist"]]] = 1

In [17]:
np.unique(train_mat)

array([  0.00000000e+00,   1.00000000e+00,   2.00000000e+00, ...,
         2.32650000e+05,   2.42328000e+05,   4.19157000e+05])

In [18]:
#transform train data
train_data = {}
for row in df_train.iterrows():
    user   = row[1]['user']
    artist = row[1]['artist']
    plays  = row[1]['plays']

    if not user in train_data:
        train_data[user] = {}

    train_data[user][artist] = int(plays)

# Compute the global median and per-user median.
user_medians = {}
for user, user_data in train_data.iteritems():
    user_plays = []
    for artist, plays in user_data.iteritems():
        user_plays.append(plays)

    user_medians[user] = np.median(np.array(user_plays))

In [19]:
#subtract out the median
user_median_matrix = np.zeros((len(unique_train_users), len(unique_train_artists)))
for user in unique_train_users:
    user_median_matrix[users_map_dict[user],:] = np.repeat(user_medians[user], len(unique_train_artists))
print user_median_matrix

[[  63.   63.   63. ...,   63.   63.   63.]
 [ 376.  376.  376. ...,  376.  376.  376.]
 [ 262.  262.  262. ...,  262.  262.  262.]
 ..., 
 [  17.   17.   17. ...,   17.   17.   17.]
 [ 133.  133.  133. ...,  133.  133.  133.]
 [  28.   28.   28. ...,   28.   28.   28.]]


In [23]:
print train_mat.shape
print user_median_matrix.shape

(233286, 2000)
(233286, 2000)


In [31]:
# subtract train matrix by user meian matrix
mat = np.array(train_mat - np.array(user_median_matrix))


In [32]:
print mat

[[ -63.  -63.  -63. ...,  -63.  -63.  -63.]
 [-376. -376. -376. ..., -376. -376. -376.]
 [-262. -262. -262. ..., -262. -262. -262.]
 ..., 
 [ -17.  -17.  -17. ...,  -17.  -17.  -17.]
 [-133. -133. -133. ..., -133. -133. -133.]
 [ -28.  -28.  -28. ...,  -28.  -28.  -28.]]


#### matrix factorization function

In [49]:
def matrix_factorization(R, ref_mat, P, Q, K, steps=5000, alpha=0.0002, beta=0.2):
    Q = Q.T
    for step in range(steps):
        print step
        for i in range(len(R)):
            for j in range(len(R[i])):
                if ref_mat[i][j] == 1:
                    eij = R[i][j] - np.dot(P[i,:],Q[:,j])
                    for k in range(K):
                        P[i][k] = P[i][k] + alpha * (2 * eij * Q[k][j] - beta * P[i][k])
                        Q[k][j] = Q[k][j] + alpha * (2 * eij * P[i][k] - beta * Q[k][j])
        eR = np.dot(P,Q)
        e = 0
        for i in range(len(R)):
            for j in range(len(R[i])):
                if ref_mat[i][j] == 1:
                    e = e + pow(R[i][j] - np.dot(P[i,:],Q[:,j]), 2)
                    for k in range(K):
                        e = e + (beta/2) * ( pow(P[i][k],2) + pow(Q[k][j],2) )
        if e < 0.001:
            break
    return P, Q.T

In [45]:
N = len(mat)
M = len(mat[0])
K = 2
P = np.random.rand(N,K)
Q = np.random.rand(M,K)

In [50]:
nP, nQ = matrix_factorization(mat, reference_mat, P, Q, K)

0


KeyboardInterrupt: 

In [ ]:
fitted_mat = np.dot(nP, nQ.T)

#### Validation accuracy